<a href="https://colab.research.google.com/github/gani0325/2022/blob/main/BOAZ/%EB%AF%B8%EB%8B%88%ED%94%8C%EC%A0%9D2%EC%9B%94/%EB%AA%A8%EB%8D%B83%EA%B0%9C%EB%8B%A4%EA%B0%99%EC%9D%B4voting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix

In [3]:
!pip install optuna

     |████████████████████████████████| 308 kB 13.3 MB/s 
     |████████████████████████████████| 81 kB 8.5 MB/s 
     |████████████████████████████████| 210 kB 72.6 MB/s 
     |████████████████████████████████| 75 kB 3.9 MB/s 
     |████████████████████████████████| 113 kB 77.9 MB/s 
     |████████████████████████████████| 49 kB 6.9 MB/s 
     |████████████████████████████████| 150 kB 58.5 MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11137 sha256=3d01e45a8d72f1e9d3706b87e7185e98581e4d6ccd58798be3f76eadcdeb1c2b
  Stored in directory: /root/.cache/pip/wheels/9f/18/84/8f69f8b08169c7bae2dde6bd7daf0c19fca8c8e500ee620a28
Successfully built pyperclip


In [4]:
import optuna
from optuna import Trial, visualization
from optuna.samplers import TPESampler
from sklearn.model_selection import cross_val_score
import warnings
warnings.filterwarnings('ignore')

# RandomForest 최종

In [5]:
test_fast = pd.read_csv("/content/drive/MyDrive/가은/BOAZ/미니플젝_2022_02/test_fast_final.csv")
test_slow = pd.read_csv("/content/drive/MyDrive/가은/BOAZ/미니플젝_2022_02/test_slow_final.csv")

train_fast = pd.read_csv("/content/drive/MyDrive/가은/BOAZ/미니플젝_2022_02/train_fast_final.csv")
train_slow = pd.read_csv("/content/drive/MyDrive/가은/BOAZ/미니플젝_2022_02/train_slow_final.csv")

In [6]:
train_fast.shape, test_fast.shape, train_slow.shape, test_slow.shape

((102316, 68), (25575, 68), (102294, 68), (25575, 68))

## 급속 충전기

In [7]:
X_fast_train = train_fast.drop(['fast_exist'], axis=1)
y_fast_train = train_fast.fast_exist

X_fast_test = test_fast.drop(['fast_exist'], axis=1)
y_fast_test = test_fast.fast_exist

In [8]:
from sklearn.feature_selection import SelectKBest, chi2

# SelctKBest k=10
select = SelectKBest(chi2, k=30)
select.fit(X_fast_train, y_fast_train)

# 선택한 특성
f_scores = pd.DataFrame()

f_scores['chi'] = select.scores_
f_scores['p'] = select.pvalues_
f_scores['support'] = select.get_support()
f_scores['attribute'] = X_fast_train.columns
f_scores[f_scores['support'] == True]


,chi,p,support,attribute
2,18210.932711,0.0,True,편의시설_scaled
4,20045.894649,0.0,True,청소년관련시설_scaled
6,21260.349824,0.0,True,차량수_scaled
7,15913.034653,0.0,True,주차장_scaled
12,23806.979725,0.0,True,인구_scaled
13,21680.643277,0.0,True,의료시설_scaled
15,18139.716003,0.0,True,위락시설_scaled
17,14709.087896,0.0,True,완속_scaled
18,19236.928835,0.0,True,업무시설_scaled
19,18655.930284,0.0,True,숙박시설_scaled


In [ ]:
# 선택한 특성만 저장
#X_train_select = select.transform(X_fast_train)
#X_test_select = select.transform(X_fast_test)

In [ ]:
#Optuna
from pandas.core.common import random_state

def RF_objective(trial):
    bootstrap = trial.suggest_categorical('bootstrap',['True','False'])
    max_depth = trial.suggest_int('max_depth', 1, 10000)
    max_features = trial.suggest_categorical('max_features', ['auto', 'sqrt','log2'])
    max_leaf_nodes = trial.suggest_int('max_leaf_nodes', 1, 10000)
    n_estimators =  trial.suggest_int('n_estimators', 30, 1000)
   
    model = RandomForestClassifier(bootstrap = bootstrap,
                                 max_depth = max_depth, max_features = max_features,
                                 max_leaf_nodes = max_leaf_nodes,n_estimators = n_estimators,n_jobs=2,random_state=25)

    
    model.fit(X_train_select30, y_fast_train)    
    y_fast_pred = model.predict(X_test_select30)
    f1 = f1_score(y_fast_test, y_fast_pred)
  

    return f1
    
#Execute optuna and set hyperparameters
RF_study = optuna.create_study(direction='maximize')
RF_study.optimize(RF_objective, n_trials=250)

#Create an instance with tuned hyperparameters
optimized_RF = RandomForestClassifier(max_depth = RF_study.best_params['max_depth'], max_leaf_nodes = RF_study.best_params['max_leaf_nodes'],
                                      n_estimators = RF_study.best_params['n_estimators'],n_jobs=2,max_features =RF_study.best_params['max_features'],
                                      bootstrap= RF_study.best_params['bootstrap'],random_state=25)

[I 2022-02-25 04:02:29,328] A new study created in memory with name: no-name-7933c02e-d93b-4264-9263-e582db1c6c9e
[I 2022-02-25 04:04:36,092] Trial 0 finished with value: 0.6190476190476191 and parameters: {'bootstrap': 'True', 'max_depth': 7609, 'max_features': 'sqrt', 'max_leaf_nodes': 8797, 'n_estimators': 216}. Best is trial 0 with value: 0.6190476190476191.
[I 2022-02-25 04:06:21,365] Trial 1 finished with value: 0.6190476190476191 and parameters: {'bootstrap': 'False', 'max_depth': 464, 'max_features': 'log2', 'max_leaf_nodes': 268, 'n_estimators': 282}. Best is trial 0 with value: 0.6190476190476191.
[I 2022-02-25 04:10:02,269] Trial 2 finished with value: 0.6511627906976745 and parameters: {'bootstrap': 'True', 'max_depth': 9697, 'max_features': 'log2', 'max_leaf_nodes': 2371, 'n_estimators': 606}. Best is trial 2 with value: 0.6511627906976745.
[I 2022-02-25 04:11:55,319] Trial 3 finished with value: 0.6190476190476191 and parameters: {'bootstrap': 'False', 'max_depth': 2889, 

In [9]:
model = RandomForestClassifier(random_state=25, n_estimators=31, max_depth=2699, max_leaf_nodes=172,bootstrap = True, max_features = 'auto')
model.fit(X_fast_train, y_fast_train)

RandomForestClassifier(max_depth=2699, max_leaf_nodes=172, n_estimators=31,
                       random_state=25)

In [10]:
train_preds = model.predict(X_fast_train)
f1 = f1_score(train_preds,y_fast_train)
f1

1.0

In [11]:
 y_fast_pred = model.predict(X_fast_test)
 f1 = f1_score(y_fast_test, y_fast_pred)
 f1

0.6666666666666665

In [12]:
y_fast_prob = model.predict_proba(X_fast_test)

In [13]:
for threshold in range(1,10):
    threshold = round(threshold*0.1, 1)
    y_fast_pred = np.where(y_fast_prob>=threshold, 1, 0)
    print(f'{threshold} :{f1_score(y_fast_test, y_fast_pred[:,1])}')

0.1 :0.44680851063829796
0.2 :0.5352112676056338
0.3 :0.6551724137931034
0.4 :0.7083333333333334
0.5 :0.6666666666666665
0.6 :0.5499999999999999
0.7 :0.48648648648648657
0.8 :0.36363636363636365
0.9 :0.21428571428571425


## tuning 후 feature selection (10~65)

In [14]:
ks = [10,15,20,25,30,35,40,45,50,55,60,65]

### fast
* 최적 score = 0.7186
* 최적 k = 60
* 최적 threshold = 0.3

In [15]:
for k in ks:
    fit = SelectKBest(chi2, k=k).fit(X_fast_train, y_fast_train)
    new_X_fast_train = fit.transform(X_fast_train)
    new_X_fast_test = fit.transform(X_fast_test)
    
    model.fit(new_X_fast_train, y_fast_train)
    y_fast_prob = model.predict_proba(new_X_fast_test)
    print(f'================================{k}================================')
    for threshold in range(1,10):
        threshold = round(threshold*0.1, 1)
        y_fast_pred = np.where(y_fast_prob>=threshold, 1, 0)
        print(f'{threshold} :{f1_score(y_fast_test, y_fast_pred[:,1])}')

================================10================================
0.1 :0.0286493860845839
0.2 :0.28571428571428575
0.3 :0.32608695652173914
0.4 :0.37037037037037035
0.5 :0.34782608695652173
0.6 :0.3333333333333333
0.7 :0.35294117647058826
0.8 :0.4
0.9 :0.45714285714285713
================================15================================
0.1 :0.4158415841584159
0.2 :0.5833333333333334
0.3 :0.6451612903225806
0.4 :0.6538461538461539
0.5 :0.6511627906976745
0.6 :0.5499999999999999
0.7 :0.5128205128205129
0.8 :0.45714285714285713
0.9 :0.47058823529411764
================================20================================
0.1 :0.4285714285714286
0.2 :0.5675675675675675
0.3 :0.6666666666666667
0.4 :0.6071428571428571
0.5 :0.56
0.6 :0.48888888888888893
0.7 :0.4444444444444444
0.8 :0.3529411764705882
0.9 :0.3125
================================25================================
0.1 :0.42
0.2 :0.5675675675675675
0.3 :0.6268656716417911
0.4 :0.6101694915254237
0.5 :0.5416666666666667
0.6 :0.428

```
================================40================================
0.4 :0.7169811320754718

================================65================================
0.4 :0.7169811320754718
```

## 완속 충전기

In [16]:
X_slow_train = train_slow.drop(['slow_exist'], axis=1)
y_slow_train = train_slow.slow_exist

X_slow_test = test_slow.drop(['slow_exist'], axis=1)
y_slow_test = test_slow.slow_exist

# **Optuna**

In [ ]:
from pandas.core.common import random_state
def RF_objective(trial):
    bootstrap = trial.suggest_categorical('bootstrap',['True','False'])
    max_depth = trial.suggest_int('max_depth', 1, 10000)
    max_features = trial.suggest_categorical('max_features', ['auto', 'sqrt','log2'])
    max_leaf_nodes = trial.suggest_int('max_leaf_nodes', 1, 10000)
    n_estimators =  trial.suggest_int('n_estimators', 30, 1000)
   
    model = RandomForestClassifier(bootstrap = bootstrap,
                                 max_depth = max_depth, max_features = max_features,
                                 max_leaf_nodes = max_leaf_nodes,n_estimators = n_estimators,n_jobs=2,random_state=25)

    
    model.fit(X_slow_train, y_slow_train)    
    y_slow_pred = model.predict(X_slow_test)
    f1 = f1_score(y_slow_test, y_slow_pred)
  

    return f1
    
#Execute optuna and set hyperparameters
RF_study = optuna.create_study(direction='maximize')
RF_study.optimize(RF_objective, n_trials=150)

#Create an instance with tuned hyperparameters
optimized_RF = RandomForestClassifier(max_depth = RF_study.best_params['max_depth'], max_leaf_nodes = RF_study.best_params['max_leaf_nodes'],
                                      n_estimators = RF_study.best_params['n_estimators'],n_jobs=2,max_features =RF_study.best_params['max_features'],
                                      bootstrap= RF_study.best_params['bootstrap'],random_state=25)

[I 2022-02-25 09:17:09,348] A new study created in memory with name: no-name-34a263bf-53d6-437b-a0c6-d32100246e86
[I 2022-02-25 09:27:42,455] Trial 0 finished with value: 0.7575757575757577 and parameters: {'bootstrap': 'True', 'max_depth': 6583, 'max_features': 'sqrt', 'max_leaf_nodes': 1497, 'n_estimators': 959}. Best is trial 0 with value: 0.7575757575757577.
[I 2022-02-25 09:32:09,893] Trial 1 finished with value: 0.7575757575757577 and parameters: {'bootstrap': 'True', 'max_depth': 5032, 'max_features': 'log2', 'max_leaf_nodes': 9436, 'n_estimators': 563}. Best is trial 0 with value: 0.7575757575757577.
[I 2022-02-25 09:39:21,114] Trial 2 finished with value: 0.746268656716418 and parameters: {'bootstrap': 'False', 'max_depth': 5094, 'max_features': 'log2', 'max_leaf_nodes': 10000, 'n_estimators': 897}. Best is trial 0 with value: 0.7575757575757577.
[I 2022-02-25 09:42:55,163] Trial 3 finished with value: 0.746268656716418 and parameters: {'bootstrap': 'True', 'max_depth': 5131, 

KeyboardInterrupt: ignored

In [17]:
model1 = RandomForestClassifier(random_state=25, n_estimators=251, max_depth=9043, max_leaf_nodes=7310,bootstrap = True, max_features = 'sqrt')
model1.fit(X_slow_train, y_slow_train)

RandomForestClassifier(max_depth=9043, max_features='sqrt', max_leaf_nodes=7310,
                       n_estimators=251, random_state=25)

In [18]:
train_preds = model1.predict( X_slow_train)
f1 = f1_score(train_preds,y_slow_train)
f1

1.0

In [20]:
y_slow_prob = model1.predict_proba(X_slow_test)

In [21]:
for threshold in range(1,10):
    threshold = round(threshold*0.1, 1)
    y_slow_pred = np.where(y_slow_prob>=threshold, 1, 0)
    print(f'{threshold} :{f1_score(y_slow_test, y_slow_pred[:,1])}')

0.1 :0.5045045045045045
0.2 :0.6923076923076923
0.3 :0.7297297297297297
0.4 :0.7123287671232877
0.5 :0.7692307692307692
0.6 :0.7540983606557377
0.7 :0.7017543859649122
0.8 :0.6122448979591837
0.9 :0.5333333333333333


## tuning 후 feature selection (10~65)

In [22]:
ks = [10,15,20,25,30,35,40,45,50,55,60,65]

### slow
* 성능 향상 X

In [23]:
for k in ks:
    fit = SelectKBest(chi2, k=k).fit(X_slow_train, y_slow_train)
    new_X_slow_train = fit.transform(X_slow_train)
    new_X_slow_test = fit.transform(X_slow_test)
    
    model1.fit(new_X_slow_train, y_slow_train)
    y_fast_prob = model1.predict_proba(new_X_slow_test)
    print(f'================================{k}================================')
    for threshold in range(1,10):
        threshold = round(threshold*0.1, 1)
        y_slow_pred = np.where(y_slow_prob>=threshold, 1, 0)
        print(f'{threshold} :{f1_score(y_slow_test, y_slow_pred[:,1])}')

================================10================================
0.1 :0.5045045045045045
0.2 :0.6923076923076923
0.3 :0.7297297297297297
0.4 :0.7123287671232877
0.5 :0.7692307692307692
0.6 :0.7540983606557377
0.7 :0.7017543859649122
0.8 :0.6122448979591837
0.9 :0.5333333333333333
================================15================================
0.1 :0.5045045045045045
0.2 :0.6923076923076923
0.3 :0.7297297297297297
0.4 :0.7123287671232877
0.5 :0.7692307692307692
0.6 :0.7540983606557377
0.7 :0.7017543859649122
0.8 :0.6122448979591837
0.9 :0.5333333333333333
================================20================================
0.1 :0.5045045045045045
0.2 :0.6923076923076923
0.3 :0.7297297297297297
0.4 :0.7123287671232877
0.5 :0.7692307692307692
0.6 :0.7540983606557377
0.7 :0.7017543859649122
0.8 :0.6122448979591837
0.9 :0.5333333333333333
================================25================================
0.1 :0.5045045045045045
0.2 :0.6923076923076923
0.3 :0.7297297297297297
0.4 :0.71232

```
급속
================================40================================
0.4 :0.7169811320754718

================================65================================
0.4 :0.7169811320754718


완속
0.5 :0.7692307692307692
```


# LightGBM_Optimized

In [24]:
from os import listdir
from os.path import isfile, join
import pandas as pd
#import geopandas as gpd
import time
import json
import folium
import matplotlib.pyplot as plt
%matplotlib inline

import numpy as np
import seaborn as sns
#import fiona
#from shapely.geometry.multipolygon import MultiPolygon
#from shapely.geometry import Point, Polygon, LineString
#from shapely import wkt

#from tqdm import trange, tqdm, tqdm_notebook, tnrange

from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from sklearn.preprocessing import Binarizer, MinMaxScaler
import sklearn.metrics as metrics
from lightgbm import LGBMClassifier
import xgboost as xgb

from imblearn.over_sampling import SMOTE
from functools import partial
#from bayes_opt import BayesianOptimization

from sklearn.feature_selection import SelectKBest, chi2
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix, accuracy_score

In [25]:
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 50)

In [26]:
train_fast_X = train_fast.iloc[:, :-1]
train_fast_y = train_fast.iloc[:, -1]
test_fast_X = test_fast.iloc[:, :-1]
test_fast_y = test_fast.iloc[:, -1]

train_slow_X = train_slow.iloc[:, :-1]
train_slow_y = train_slow.iloc[:, -1]
test_slow_X = test_slow.iloc[:, :-1]
test_slow_y = test_slow.iloc[:, -1]

In [27]:
lgbm_clf_fast = LGBMClassifier(n_jobs=-1)
lgbm_clf_slow = LGBMClassifier(n_jobs=-1)


In [28]:
model_fast = [lgbm_clf_fast]
model_slow = [lgbm_clf_slow]

In [29]:
for x in model_fast:
    x.fit(train_fast_X, train_fast_y)
for x in model_slow:
    x.fit(train_slow_X, train_slow_y)

In [30]:
thresholds = np.arange(0,1,0.1)

for x in model_fast:
    print(x)
    pred_proba=x.predict_proba(np.array(test_fast_X))
    pred_proba_1 = pred_proba[:,1].reshape(-1,1)
    for threshold in thresholds:
        binarizer = Binarizer(threshold=threshold)
        model_cl_pred = binarizer.transform(pred_proba_1)
        print('threshold : ', threshold)
        print('오차 행렬')
        confusion = confusion_matrix(test_fast_y,model_cl_pred)
        print(confusion)
        print('accuracy : ', accuracy_score(test_fast_y,model_cl_pred))
        print('precision : ', precision_score(test_fast_y,model_cl_pred))
        print('recall : ', recall_score(test_fast_y,model_cl_pred))
        print('f1_score : ', f1_score(test_fast_y,model_cl_pred))
        print('')
    print('===================================================')
    print('')

LGBMClassifier()
threshold :  0.0
오차 행렬
[[    0 25550]
 [    0    25]]
accuracy :  0.0009775171065493646
precision :  0.0009775171065493646
recall :  1.0
f1_score :  0.001953125

threshold :  0.1
오차 행렬
[[25534    16]
 [    6    19]]
accuracy :  0.9991397849462366
precision :  0.5428571428571428
recall :  0.76
f1_score :  0.6333333333333332

threshold :  0.2
오차 행렬
[[25536    14]
 [    7    18]]
accuracy :  0.9991788856304985
precision :  0.5625
recall :  0.72
f1_score :  0.631578947368421

threshold :  0.30000000000000004
오차 행렬
[[25539    11]
 [    8    17]]
accuracy :  0.9992570869990225
precision :  0.6071428571428571
recall :  0.68
f1_score :  0.6415094339622641

threshold :  0.4
오차 행렬
[[25540    10]
 [    8    17]]
accuracy :  0.9992961876832844
precision :  0.6296296296296297
recall :  0.68
f1_score :  0.6538461538461539

threshold :  0.5
오차 행렬
[[25543     7]
 [    9    16]]
accuracy :  0.9993743890518084
precision :  0.6956521739130435
recall :  0.64
f1_score :  0.6666666666666666

In [31]:
thresholds = np.arange(0,1,0.1)

for x in model_slow:
    print(x)
    pred_proba=x.predict_proba(np.array(test_slow_X))
    pred_proba_1 = pred_proba[:,1].reshape(-1,1)
    for threshold in thresholds:
        binarizer = Binarizer(threshold=threshold)
        model_cl_pred = binarizer.transform(pred_proba_1)
        print('threshold : ', threshold)
        print('오차 행렬')
        confusion = confusion_matrix(test_slow_y,model_cl_pred)
        print(confusion)
        print('accuracy : ', accuracy_score(test_slow_y,model_cl_pred))
        print('precision : ', precision_score(test_slow_y,model_cl_pred))
        print('recall : ', recall_score(test_slow_y,model_cl_pred))
        print('f1_score : ', f1_score(test_slow_y,model_cl_pred))
        print('')
    print('===================================================')
    print('')

LGBMClassifier()
threshold :  0.0
오차 행렬
[[    0 25545]
 [    0    30]]
accuracy :  0.0011730205278592375
precision :  0.0011730205278592375
recall :  1.0
f1_score :  0.0023432923257176333

threshold :  0.1
오차 행렬
[[25520    25]
 [    1    29]]
accuracy :  0.9989833822091887
precision :  0.5370370370370371
recall :  0.9666666666666667
f1_score :  0.6904761904761905

threshold :  0.2
오차 행렬
[[25521    24]
 [    1    29]]
accuracy :  0.9990224828934506
precision :  0.5471698113207547
recall :  0.9666666666666667
f1_score :  0.6987951807228916

threshold :  0.30000000000000004
오차 행렬
[[25523    22]
 [    1    29]]
accuracy :  0.9991006842619746
precision :  0.5686274509803921
recall :  0.9666666666666667
f1_score :  0.7160493827160493

threshold :  0.4
오차 행렬
[[25527    18]
 [    2    28]]
accuracy :  0.9992179863147606
precision :  0.6086956521739131
recall :  0.9333333333333333
f1_score :  0.7368421052631579

threshold :  0.5
오차 행렬
[[25530    15]
 [    2    28]]
accuracy :  0.999335288367546

In [32]:
fea_imp_ = pd.DataFrame({'cols':train_fast_X.columns, 'fea_imp':lgbm_clf_fast.feature_importances_})
fea_imp_.sort_values(by=['fea_imp'], ascending=True)

,cols,fea_imp
44,dw_lanes_5_scaled,0
42,up_lanes_5_scaled,0
41,up_lanes_6_scaled,0
62,유원지,0
40,width_5_scaled,0
...,...,...
64,체력증진시설,97
8,주거지_scaled,98
59,급속,135
25,면적_scaled,154


In [33]:
import pandas as pd
import numpy as np
import optuna
from optuna import Trial, visualization
from optuna.samplers import TPESampler
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix

In [34]:
X_fast_train = train_fast.iloc[:, :-1]
y_fast_train = train_fast.iloc[:, -1]
X_fast_test = test_fast.iloc[:, :-1]
y_fast_test = test_fast.iloc[:, -1]

X_slow_train = train_slow.iloc[:, :-1]
y_slow_train = train_slow.iloc[:, -1]
X_slow_test = test_slow.iloc[:, :-1]
y_slow_test = test_slow.iloc[:, -1]

In [35]:
param = {'learning_rate': 0.24126919501069777, 'n_estimators': 113, 'max_depth': 31, 'boosting': 'gbdt',
         'feature_fraction': 0.8637387162156923, 'scale_pos_weight': 1.6314529692749955, 'num_leaves': 33, 'bagging_freq': 16,
         'bagging_fraction': 0.8965246003392066, 'min_data_in_leaf': 890,
         'objective':'binary', 'n_jobs':-1, 'random_state':42, 'subsample':1, 'metric':'binary_logloss'}

* 급속

In [36]:
optimized_LGBM_f = LGBMClassifier(**param)

In [37]:
optimized_LGBM_f.fit(X_fast_train, y_fast_train)

LGBMClassifier(bagging_fraction=0.8965246003392066, bagging_freq=16,
               boosting='gbdt', feature_fraction=0.8637387162156923,
               learning_rate=0.24126919501069777, max_depth=31,
               metric='binary_logloss', min_data_in_leaf=890, n_estimators=113,
               num_leaves=33, objective='binary', random_state=42,
               scale_pos_weight=1.6314529692749955, subsample=1)

In [38]:
y_fast_prob = optimized_LGBM_f.predict_proba(X_fast_test)

In [39]:
y_fast_pred = optimized_LGBM_f.predict(X_fast_test)
f1_score(y_fast_test, y_fast_pred)

0.8000000000000002

In [40]:
for threshold in range(1,10):
    threshold = round(threshold*0.1, 1)
    y_fast_pred = np.where(y_fast_prob>=threshold, 1, 0)
    print(f'{threshold} :{f1_score(y_fast_test, y_fast_pred[:,1])}')

0.1 :0.7407407407407408
0.2 :0.7843137254901961
0.3 :0.7843137254901961
0.4 :0.7843137254901961
0.5 :0.8000000000000002
0.6 :0.7755102040816326
0.7 :0.7916666666666667
0.8 :0.7916666666666667
0.9 :0.7391304347826089


In [41]:
ks = [30,35,40,45,50,55,60,65]

In [ ]:
for k in ks:
    fit = SelectKBest(chi2, k=k).fit(X_fast_train, y_fast_train)
    new_X_fast_train = fit.transform(X_fast_train)
    new_X_fast_test = fit.transform(X_fast_test)
    
    optimized_LGBM_f.fit(new_X_fast_train, y_fast_train)
    y_fast_prob = optimized_LGBM_f.predict_proba(new_X_fast_test)
    print(f'================================{k}================================')
    for threshold in range(1,10):
        threshold = round(threshold*0.1, 1)
        y_fast_pred = np.where(y_fast_prob>=threshold, 1, 0)
        print(f'{threshold} :{f1_score(y_fast_test, y_fast_pred[:,1])}')

================================10================================
0.1 :0.027201145311381535
0.2 :0.027377521613832854
0.3 :0.3564356435643564
0.4 :0.3636363636363637
0.5 :0.3655913978494624
0.6 :0.40963855421686746
0.7 :0.4266666666666667
0.8 :0.4444444444444445
0.9 :0.4193548387096775
================================15================================
0.1 :0.6666666666666666
0.2 :0.6666666666666666
0.3 :0.6909090909090909
0.4 :0.7037037037037037
0.5 :0.6792452830188679
0.6 :0.6923076923076923
0.7 :0.6938775510204083
0.8 :0.6938775510204083
0.9 :0.6666666666666666
================================20================================
0.1 :0.7037037037037037
0.2 :0.7450980392156863
0.3 :0.7450980392156863
0.4 :0.7346938775510204
0.5 :0.7346938775510204
0.6 :0.7346938775510204
0.7 :0.7346938775510204
0.8 :0.7659574468085107
0.9 :0.7659574468085107
================================25================================
0.1 :0.6122448979591836
0.2 :0.6122448979591836
0.3 :0.6382978723404256
0.4 :0.

* 최적 score = 0.7169811320754718
* 최적 k = 40
* 최적 threshold = 0.5, 0.6


왜 나는
* 최적 score = 0.6 :0.7916666666666667
* 최적 k = 65
* 최적 threshold = 0.6, 0.7

* 완속

In [42]:
param = {'learning_rate': 0.24126919501069777, 'n_estimators': 113, 'max_depth': 31,
         'boosting': 'gbdt', 'feature_fraction': 0.8637387162156923, 'scale_pos_weight': 1.6314529692749955,
         'num_leaves': 33, 'bagging_freq': 16, 'bagging_fraction': 0.8965246003392066, 'min_data_in_leaf': 890,
         'objective':'binary', 'n_jobs':-1, 'random_state':42, 'subsample':1, 'metric':'binary_logloss'}

In [43]:
optimized_LGBM_s = LGBMClassifier(**param)


In [44]:
optimized_LGBM_s.fit(X_slow_train, y_slow_train)

LGBMClassifier(bagging_fraction=0.8965246003392066, bagging_freq=16,
               boosting='gbdt', feature_fraction=0.8637387162156923,
               learning_rate=0.24126919501069777, max_depth=31,
               metric='binary_logloss', min_data_in_leaf=890, n_estimators=113,
               num_leaves=33, objective='binary', random_state=42,
               scale_pos_weight=1.6314529692749955, subsample=1)

In [45]:
y_slow_prob = optimized_LGBM_s.predict_proba(X_slow_test)

In [46]:
y_slow_pred = optimized_LGBM_s.predict(X_slow_test)
f1_score(y_slow_test, y_slow_pred)

0.767123287671233

In [47]:
for threshold in range(1,10):
    threshold = round(threshold*0.1, 1)
    y_slow_pred = np.where(y_slow_prob>=threshold, 1, 0)
    print(f'{threshold} :{f1_score(y_slow_test, y_slow_pred[:,1])}')

0.1 :0.7435897435897435
0.2 :0.7435897435897435
0.3 :0.7466666666666667
0.4 :0.767123287671233
0.5 :0.767123287671233
0.6 :0.7777777777777778
0.7 :0.7887323943661972
0.8 :0.7887323943661972
0.9 :0.8


In [48]:
for k in ks:
    fit = SelectKBest(chi2, k=k).fit(X_slow_train, y_slow_train)
    new_X_slow_train = fit.transform(X_slow_train)
    new_X_slow_test = fit.transform(X_slow_test)
    
    optimized_LGBM_s.fit(new_X_slow_train, y_slow_train)
    y_slow_prob = optimized_LGBM_s.predict_proba(new_X_slow_test)
    print(f'================================{k}================================')
    for threshold in range(1,10):
        threshold = round(threshold*0.1, 1)
        y_slow_pred = np.where(y_slow_prob>=threshold, 1, 0)
        print(f'{threshold} :{f1_score(y_slow_test, y_slow_pred[:,1])}')

================================30================================
0.1 :0.7341772151898733
0.2 :0.7200000000000001
0.3 :0.7200000000000001
0.4 :0.7027027027027029
0.5 :0.7323943661971831
0.6 :0.7323943661971831
0.7 :0.7323943661971831
0.8 :0.7536231884057971
0.9 :0.7761194029850748
================================35================================
0.1 :0.6987951807228916
0.2 :0.7160493827160493
0.3 :0.7012987012987012
0.4 :0.7397260273972602
0.5 :0.7397260273972602
0.6 :0.7397260273972602
0.7 :0.7605633802816902
0.8 :0.7536231884057971
0.9 :0.7761194029850748
================================40================================
0.1 :0.7341772151898733
0.2 :0.7435897435897435
0.3 :0.763157894736842
0.4 :0.7733333333333334
0.5 :0.7733333333333334
0.6 :0.7733333333333334
0.7 :0.7837837837837838
0.8 :0.7887323943661972
0.9 :0.8115942028985509
================================45================================
0.1 :0.7160493827160493
0.2 :0.7435897435897435
0.3 :0.7532467532467533
0.4 :0.763157

* 최적 score = 0.7945205479452055
* 최적 k = 55
* 최적 threshold = 0.5

왜 나는
* 최적 score = 0.835820895522388
* 최적 k = 65
* 최적 threshold = 0.9

# XGBClassifier

In [49]:
import pandas as pd
import numpy as np
from xgboost import XGBClassifier
import optuna
from optuna import Trial, visualization
from optuna.samplers import TPESampler
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix, accuracy_score

In [50]:
import os, random # random seed 고정 
def set_seeds(seed): 
    os.environ['PYTHONHASHSEED'] = str(seed) 
    random.seed(seed) 
    np.random.seed(seed) 
    # tf.random.set_seed(seed) # Tensorflow 사용시 
    
SEED = 555
set_seeds(SEED)

In [51]:
XGB = XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1,
              n_estimators=100, n_jobs=-1, num_parallel_tree=1, random_state=42,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1)


* 급속

In [ ]:
XGB.fit(X_fast_train, y_fast_train)
y_fast_prob = XGB.predict_proba(X_fast_test)

for threshold in range(1,10):
    threshold = round(threshold*0.1, 1)
    y_fast_pred = np.where(y_fast_prob>=threshold, 1, 0)
    print(f'{threshold} :{f1_score(y_fast_test, y_fast_pred[:,1])}')

0.1 :0.6086956521739131
0.2 :0.6885245901639344
0.3 :0.6909090909090909
0.4 :0.7037037037037037
0.5 :0.6923076923076923
0.6 :0.6666666666666666
0.7 :0.6956521739130435
0.8 :0.6666666666666665
0.9 :0.6666666666666665


In [ ]:
ks = [10,15,20,25,30,35,40,45,50,55,60,65]            # ## tuning 후 feature selection (10~65)

In [ ]:
for k in ks:
    fit = SelectKBest(chi2, k=k).fit(X_fast_train, y_fast_train)
    new_X_fast_train = fit.transform(X_fast_train)
    new_X_fast_test = fit.transform(X_fast_test)
    
    XGB.fit(new_X_fast_train, y_fast_train)
    y_fast_prob = XGB.predict_proba(new_X_fast_test)
    print(f'================================{k}================================')
    for threshold in range(1,10):
        threshold = round(threshold*0.1, 1)
        y_fast_pred = np.where(y_fast_prob>=threshold, 1, 0)
        print(f'{threshold} :{f1_score(y_fast_test, y_fast_pred[:,1])}')

================================10================================
0.1 :0.027027027027027025
0.2 :0.43902439024390244
0.3 :0.5070422535211268
0.4 :0.5964912280701754
0.5 :0.6071428571428571
0.6 :0.6296296296296295
0.7 :0.5882352941176471
0.8 :0.5531914893617023
0.9 :0.5652173913043478
================================15================================
0.1 :0.5185185185185186
0.2 :0.6060606060606061
0.3 :0.6440677966101696
0.4 :0.6206896551724138
0.5 :0.576923076923077
0.6 :0.5490196078431373
0.7 :0.5416666666666667
0.8 :0.5531914893617023
0.9 :0.5454545454545454
================================20================================
0.1 :0.5633802816901409
0.2 :0.5806451612903226
0.3 :0.6428571428571428
0.4 :0.6792452830188679
0.5 :0.6538461538461539
0.6 :0.68
0.7 :0.7083333333333334
0.8 :0.6521739130434783
0.9 :0.5853658536585366
================================25================================
0.1 :0.6349206349206348
0.2 :0.6666666666666666
0.3 :0.7058823529411765
0.4 :0.72
0.5 :0.7346938

- 60
- 0.4 :0.7777777777777777

* 완속

In [52]:
XGB.fit(X_slow_train, y_slow_train)
y_slow_prob = XGB.predict_proba(X_slow_test)

for threshold in range(1,10):
    threshold = round(threshold*0.1, 1)
    y_slow_pred = np.where(y_slow_prob>=threshold, 1, 0)
    print(f'{threshold} :{f1_score(y_slow_test, y_slow_pred[:,1])}')

0.1 :0.6451612903225806
0.2 :0.6976744186046512
0.3 :0.6904761904761905
0.4 :0.7160493827160493
0.5 :0.7272727272727273
0.6 :0.767123287671233
0.7 :0.7714285714285714
0.8 :0.7575757575757577
0.9 :0.7419354838709677


In [53]:
for k in ks:
    fit = SelectKBest(chi2, k=k).fit(X_slow_train, y_slow_train)
    new_X_slow_train = fit.transform(X_slow_train)
    new_X_slow_test = fit.transform(X_slow_test)
    
    XGB.fit(new_X_slow_train, y_slow_train)
    y_slow_prob = XGB.predict_proba(new_X_slow_test)
    print(f'================================{k}================================')
    for threshold in range(1,10):
        threshold = round(threshold*0.1, 1)
        y_slow_pred = np.where(y_slow_prob>=threshold, 1, 0)
        print(f'{threshold} :{f1_score(y_slow_test, y_slow_pred[:,1])}')

================================30================================
0.1 :0.5742574257425742
0.2 :0.5977011494252873
0.3 :0.6190476190476191
0.4 :0.6419753086419753
0.5 :0.65
0.6 :0.6133333333333334
0.7 :0.6197183098591549
0.8 :0.6470588235294117
0.9 :0.65625
================================35================================
0.1 :0.5742574257425742
0.2 :0.6236559139784946
0.3 :0.6428571428571429
0.4 :0.6419753086419753
0.5 :0.6666666666666667
0.6 :0.6666666666666667
0.7 :0.676056338028169
0.8 :0.6857142857142857
0.9 :0.6769230769230768
================================40================================
0.1 :0.5858585858585859
0.2 :0.6292134831460674
0.3 :0.6588235294117647
0.4 :0.6913580246913581
0.5 :0.7088607594936709
0.6 :0.7105263157894737
0.7 :0.676056338028169
0.8 :0.6666666666666667
0.9 :0.696969696969697
================================45================================
0.1 :0.6451612903225806
0.2 :0.6590909090909091
0.3 :0.6744186046511628
0.4 :0.725
0.5 :0.7435897435897435
0.6 :

- 65
- 0.9 :0.7936507936507938

# 앙상블
## 1) Voting

In [54]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, ExtraTreesClassifier, BaggingClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegressionCV, RidgeClassifier
# VotingClassifier
from sklearn.ensemble import VotingClassifier

from sklearn.model_selection import cross_validate
from sklearn.model_selection import train_test_split

In [55]:
train_fast_X = train_fast.iloc[:, :-1]
train_fast_y = train_fast.iloc[:, -1]
test_fast_X = test_fast.iloc[:, :-1]
test_fast_y = test_fast.iloc[:, -1]

train_slow_X = train_slow.iloc[:, :-1]
train_slow_y = train_slow.iloc[:, -1]
test_slow_X = test_slow.iloc[:, :-1]
test_slow_y = test_slow.iloc[:, -1]

In [63]:
#VotingClassifier 

from sklearn.ensemble import VotingClassifier

soft_vote = VotingClassifier(estimators=[('rfc', model),
    ('lgbm', optimized_LGBM_f),
    ('xgb',XGB) ], voting='soft')

In [64]:
soft_vote.fit(train_fast_X,train_fast_y)
soft_vote.score(test_fast_X,test_fast_y)

0.9995698924731182

- 급속 soft voting

In [65]:
# 모델 비교
from sklearn.metrics import classification_report, confusion_matrix
for models in (model,optimized_LGBM_f, XGB, soft_vote):
  models.fit(train_fast_X,train_fast_y)
  y_pred = models.predict(test_fast_X)
  print(models.__class__.__name__," : ",accuracy_score(test_fast_y,y_pred))
  

  f1 = f1_score(test_fast_y, y_pred)
  print(f1)

RandomForestClassifier  :  0.9994134897360704
0.6666666666666665
LGBMClassifier  :  0.9996089931573803
0.8000000000000002
XGBClassifier  :  0.9993743890518084
0.6923076923076923
VotingClassifier  :  0.9995698924731182
0.7755102040816326


- 완속 soft voting

In [66]:
#VotingClassifier 

from sklearn.ensemble import VotingClassifier

soft_vote_slow = VotingClassifier(estimators=[('rfc', model1),
    ('lgbm', optimized_LGBM_s),
    ('xgb',XGB) ], voting='soft')

In [67]:
soft_vote_slow.fit(train_slow_X,train_slow_y)
soft_vote_slow.score(test_slow_X,test_slow_y)

0.9992961876832844

In [68]:
# 모델 비교
from sklearn.metrics import classification_report, confusion_matrix
for models in (model1,optimized_LGBM_s, XGB, soft_vote_slow):
  models.fit(train_slow_X,train_slow_y)
  y_pred = models.predict(test_slow_X)
  print(models.__class__.__name__," : ",accuracy_score(test_slow_y,y_pred))
  

  f1 = f1_score(test_slow_y, y_pred)
  print(f1)

RandomForestClassifier  :  0.9994134897360704
0.7692307692307692
LGBMClassifier  :  0.9993352883675465
0.767123287671233
XGBClassifier  :  0.9991788856304985
0.7272727272727273
VotingClassifier  :  0.9992961876832844
0.7567567567567568


- 급속 hard voting

In [69]:
#VotingClassifier 

from sklearn.ensemble import VotingClassifier

hard_vote_fast = VotingClassifier(estimators=[('rfc', model),
    ('lgbm', optimized_LGBM_f),
    ('xgb',XGB) ], voting='hard')

In [71]:
hard_vote_fast.fit(train_fast_X,train_fast_y)
hard_vote_fast.score(test_fast_X,test_fast_y)

0.9994916911045943

In [72]:
# 모델 비교
from sklearn.metrics import classification_report, confusion_matrix
for models in (model,optimized_LGBM_f, XGB, hard_vote_fast):
  models.fit(train_fast_X,train_fast_y)
  y_pred = models.predict(test_fast_X)
  print(models.__class__.__name__," : ",accuracy_score(test_fast_y,y_pred))

  f1 = f1_score(test_fast_y, y_pred)
  print(f1)

RandomForestClassifier  :  0.9994134897360704
0.6666666666666665
LGBMClassifier  :  0.9996089931573803
0.8000000000000002
XGBClassifier  :  0.9993743890518084
0.6923076923076923
VotingClassifier  :  0.9994916911045943
0.7346938775510204


- 완속 hard voting

In [73]:
#VotingClassifier 

from sklearn.ensemble import VotingClassifier

hard_vote_slow = VotingClassifier(estimators=[('rfc', model1),
    ('lgbm', optimized_LGBM_s),
    ('xgb',XGB) ], voting='hard')

In [74]:
hard_vote_slow.fit(train_slow_X,train_slow_y)
hard_vote_slow.score(test_slow_X,test_slow_y)

0.9993352883675465

In [75]:
# 모델 비교
from sklearn.metrics import classification_report, confusion_matrix
for models in (model1,optimized_LGBM_s, XGB, hard_vote_slow):
  models.fit(train_slow_X,train_slow_y)
  y_pred = models.predict(test_slow_X)
  print(models.__class__.__name__," : ",accuracy_score(test_slow_y,y_pred))

  f1 = f1_score(test_slow_y, y_pred)
  print(f1)

RandomForestClassifier  :  0.9994134897360704
0.7692307692307692
LGBMClassifier  :  0.9993352883675465
0.767123287671233
XGBClassifier  :  0.9991788856304985
0.7272727272727273
VotingClassifier  :  0.9993352883675465
0.767123287671233


## 2) baggingclassifier

In [ ]:
from sklearn.ensemble import BaggingClassifier
# 모델 구축
# BaggingClassifier에서 사용한 분류기가 클래스 확률추정(predict_proba)이 가능하면 자동으로 간접 투표 사용 
bag_model = BaggingClassifier(
    RandomForestClassifier(random_state=25, n_estimators=31, max_depth=2699, max_leaf_nodes=172,bootstrap = True, max_features = 'auto'),
    n_estimators=500, # 약한 학습기(결정 트리) 500개 생성
    max_samples=0.05, # 0.0~1.0 사이 실수 선택(실수 x 샘플 수) 혹은 샘플수 지정
    bootstrap=True, # True : 배깅, False : 페이스팅
    n_jobs=-1 # 훈련과 예측에 사용할 CPU 코어 수 (-1 : 가용한 모든 코어 사용)
)


In [ ]:

# 모델 학습
bag_model.fit(train_fast_X,train_fast_y)

# 모델 예측
y_pred = bag_model.predict(test_fast_X)

# 모델 평가
print(bag_model.__class__.__name__," : ",accuracy_score(test_fast_y,y_pred))

f1 = f1_score(test_fast_y, y_pred)
print(f1)

BaggingClassifier  :  0.9942521994134897
0.2054054054054054


In [ ]:
from sklearn.ensemble import BaggingClassifier
# 모델 구축
# BaggingClassifier에서 사용한 분류기가 클래스 확률추정(predict_proba)이 가능하면 자동으로 간접 투표 사용 
bag_model1 = BaggingClassifier(
    XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1,
              n_estimators=100, n_jobs=-1, num_parallel_tree=1, random_state=42,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1),
    bootstrap=True, # True : 배깅, False : 페이스팅
    n_jobs=-1 # 훈련과 예측에 사용할 CPU 코어 수 (-1 : 가용한 모든 코어 사용)
)


# 모델 학습
bag_model1.fit(train_fast_X,train_fast_y)

# 모델 예측
y_pred = bag_model1.predict(test_fast_X)

# 모델 평가
print(bag_model1.__class__.__name__," : ",accuracy_score(test_fast_y,y_pred))

f1 = f1_score(test_fast_y, y_pred)
print(f1)

BaggingClassifier  :  0.9993743890518084
0.7142857142857142


In [ ]:
from sklearn.ensemble import BaggingClassifier
# 모델 구축
# BaggingClassifier에서 사용한 분류기가 클래스 확률추정(predict_proba)이 가능하면 자동으로 간접 투표 사용 
bag_model2 = BaggingClassifier(
    LGBMClassifier(n_jobs=-1),

    bootstrap=True, # True : 배깅, False : 페이스팅

)


# 모델 학습
bag_model2.fit(train_fast_X,train_fast_y)

# 모델 예측
y_pred = bag_model2.predict(test_fast_X)

# 모델 평가
print(bag_model2.__class__.__name__," : ",accuracy_score(test_fast_y,y_pred))

f1 = f1_score(test_fast_y, y_pred)
print(f1)

BaggingClassifier  :  0.9992961876832844
0.64
